<a href="https://colab.research.google.com/github/custerc/miscellaneous-marketing-code/blob/main/Blog_Traffic_Drop_Analysis_for_Cockroach_Labs_V2_(GA4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is this?

This script is built to help analyze comparative traffic data from GA4 exports. By default, Google Analytics doesn't let you sort by how much a posts traffic has changed, making it difficult to easily locate the posts that have gained or lost the most traffic between the time periods you're comparing. This script will do that for you.

Specifically, it's designed to work with an export of the **Reports > Engagement > Landing page** report in GA4, with comparison of two time periods turned on. It can probably be adapted to work with other reports, but you will have to adjust the code. (Please make a copy and add your changes to the copy, not here in the original.)

## How do I use it?

I have made a short video walking through it, which you can see here:

VID TKTK

Alternatively, **there are written instructions below the main code blocks.**

In [3]:
# Please make a copy and work there if you want to change any code in this block.
# If you're just running the script, all you need to do is click the play button on the left.

import pandas as pd
import csv

#create empty lists for data from the current and previous periods in the export
current_list = []
previous_list = []

#open the csv file
with open (str('data-export.csv'), 'r') as file:
  data_reader=csv.reader(file)
  datalist = list(data_reader)

# Flag to determine whether to add rows to 'current' or 'previous'
add_to_current = True

for row in datalist[9:]:
# handle the empty rows and second round of dates
  if not row:
    continue
  if 'Start date' in row[0]:
    continue
  if 'End date' in row[0]:
    continue

  # keep add_to_current true until you hit # All Users
  if row[0] == '# All Users':
    add_to_current = False
    continue  # Skip the row containing '# All Users'

  # appending to the appropriate lists
  if add_to_current:
    current_list.append(row)
  else:
    previous_list.append(row)

# create two dataframes with the two lists
df_current = pd.DataFrame(current_list,columns=['Landing page', 'Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue'])
df_previous = pd.DataFrame(previous_list,columns=['Landing page', 'Sessions_prev', 'Users_prev', 'New users_prev', 'Average engagement time per session_prev', 'Conversions_prev', 'Total revenue_prev'])

# merge the dataframes and clean the data
joined = pd.merge(df_current, df_previous, on='Landing page', how='outer')
joined.replace(',','', regex=True, inplace=True)
joined.replace('%','', regex=True, inplace=True)
joined = joined.drop(index=0)

# convert to numeric datatypes
joined[['Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue', 'Sessions_prev', 'Users_prev', 'New users_prev', 'Average engagement time per session_prev', 'Conversions_prev', 'Total revenue_prev']] = joined[['Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue', 'Sessions_prev', 'Users_prev', 'New users_prev', 'Average engagement time per session_prev', 'Conversions_prev', 'Total revenue_prev']].apply(pd.to_numeric)

# create new columns tracking the percentage and raw change in sessions in the joined df
joined['SessionsDelta_perc'] = (joined['Sessions'] - joined['Sessions_prev']) / joined['Sessions_prev'] * 100
joined['SessionsDelta_raw'] = (joined['Sessions'] - joined['Sessions_prev'])
joined['EngagementDelta_perc'] = (joined['Average engagement time per session'] - joined['Average engagement time per session_prev']) / joined['Average engagement time per session_prev'] * 100
joined['EngagementDelta_raw'] = (joined['Average engagement time per session'] - joined['Average engagement time per session_prev'])
joined['ConversionsDelta_perc'] = (joined['Conversions'] - joined['Conversions_prev']) / joined['Conversions_prev'] * 100
joined['ConversionsDelta_raw'] = (joined['Conversions'] - joined['Conversions_prev'])
joined = joined.round({'SessionsDelta_perc': 1})
joined = joined.round({'EngagementDelta_perc': 1})
joined = joined.round({'ConversionsDelta_perc': 1})

In [4]:
# !!! CUSTOMIZATION OPTIONS HERE !!!
# Set minimum number of sessions required (in both current and previous periods) to return data in chart
session_floor = 50
# Set number of rows to output
rows_to_output = 2
# !!! END OF CUSTOMIZATION BLOCK !!!

# default column sorting
sortbycol = 'SessionsDelta_perc'
output = joined.sort_values(by=[sortbycol], ascending=True)

# setting session count floor
output = output.drop(output[output.Sessions_prev < session_floor].index)
output = output.drop(output[output.Sessions < session_floor].index)

#define and generate output
output = output[['Landing page', 'SessionsDelta_perc', 'SessionsDelta_raw', 'Sessions', 'Sessions_prev', 'EngagementDelta_perc', 'EngagementDelta_raw', 'ConversionsDelta_perc', 'ConversionsDelta_raw', 'Average engagement time per session', 'Average engagement time per session_prev']]
output.head(rows_to_output)

,Landing page,SessionsDelta_perc,SessionsDelta_raw,Sessions,Sessions_prev,EngagementDelta_perc,EngagementDelta_raw,ConversionsDelta_perc,ConversionsDelta_raw,Average engagement time per session,Average engagement time per session_prev
104,/blog/serverless-free,-71.3,-194,78,272,72.0,36.524793,inf,16,87.256410,50.731618
90,/blog/5-fintech-companies-refused-to-sacrifice...,-63.9,-163,92,255,36.8,5.674297,-83.3,-5,21.097826,15.423529


## Written instructions
## 1. Get the data from GA4

In GA4, navigate to Reports > Engagement > Landing page. Add any filters you would like, and turn on comparison by clicking the dates in the top right and adding the period you'd like to compare to.

Then, click the share icon (also top right) and select Download file > Download CSV.

This should download a file called `data-export.csv`. If the file has some other name, please rename if `data-export.csv`.

## 2. Upload the file to Google Colab

On the left sidebar of the screen on this page, you will see a folder icon. Click on it, and it will open a tab called "files". Drag your `data-export.csv` file into this tab to upload it to Google Colab. When this is done successully you should see `data-export.csv` in that tab listed right under a folder called `sample_data`.

## 3. Customize your output

In the second code block, under where it says `#CUSTOMIZATION OPTIONS HERE`, there are several values you can tweak to adjust the output of the code. They are:

* `session_floor` - set a minimum number of sessions; pages with fewer sessions than this during either time period you're looking at will not be displayed. Useful for filtering out low-traffic posts you don't care about. You must set this as equal to a number.
* `rows_to_output` - set the number of results you want to see in the output.

## 4. Run the code

Press the "play" button in the code blocks one at a time, starting from the top and working down. Each code block must be complete before you run the next one, or it won't work.

Note that the output is sorted by the percentage change in sessions by default, but you can use the UI to sort by other columns (see the video for details).

_Note: Over time I may add additional code blocks below these instructions that provide supplementary outputs. These may or may not be useful._